In [6]:
import yfinance as yf
import time
import warnings
warnings.filterwarnings("ignore")


import sys, os
sys.path.append(os.path.abspath(".."))

from pantheon.config import RAW_DATA_DIR, TICKERS, START_DATE, END_DATE

# Create the main output directory
output_dir = "tech_rawdata"

2025-10-06 22:11:54.674 | INFO     | pantheon.config:<module>:11 - PROJ_ROOT path is: C:\Users\Lenovo\Documents\GitHub\pantheon-quant


In [ ]:
os.makedirs(RAW_DATA_DIR / output_dir, exist_ok=True)


# Função para limpar ticker
def clean_ticker(ticker):
    return ticker.replace(".SA", "").upper()

# Loop por ticker
for ticker in TICKERS:
    try:
        print(f"📥 Downloading data for {ticker}...")

        # Download com yfinance
        df = yf.download(ticker, start=START_DATE, end=END_DATE, interval="1d")

        if df.empty:
            print(f"⚠️ No data found for {ticker}. Skipping.\n")
            continue

        # Resetar índice (Date vira coluna)
        df.reset_index(inplace=True)

        # Se houver MultiIndex nas colunas, achatar
        df.columns = [
            '_'.join(col).strip() if isinstance(col, tuple) else col
            for col in df.columns
        ]

        # Remover o ticker e o ".SA" dos nomes das colunas
        df.columns = [
            col.replace(f"_{ticker}", "").replace(".SA", "")
            for col in df.columns
        ]

        # Corrigir o nome da coluna Date_
        df.columns = [col.replace("Date_", "Date") for col in df.columns]

        # Caminho do arquivo CSV
        filepath = RAW_DATA_DIR / output_dir / f"{ticker.replace('.SA','')}.csv"

        # Salvar CSV
        df.to_csv(filepath, index=False)

        print(f"✅ Data for {ticker} saved to: {filepath}\n")

        time.sleep(5)

    except Exception as e:
        print(f"❌ Failed to download {ticker}: {e}\n")

print("\n🏁 Done! All files have been successfully generated.")


[*********************100%***********************]  1 of 1 completed

📥 Downloading data for BBAS3.SA...
✅ Data for BBAS3.SA saved to: C:\Users\User\Documents\GitHub\pantheon-quant\data\raw\stocks_rawdata_10y\BBAS3.csv



KeyboardInterrupt: 

In [14]:
import pandas as pd

dfs = []
for ticker in TICKERS:
    ticker = ticker.replace('.SA','')
    df = pd.read_csv(RAW_DATA_DIR / output_dir / f"{ticker}.csv")
    df['Ticker'] = ticker
    dfs.append(df)

In [18]:
df_unico = pd.concat(dfs, ignore_index=True)
df_unico

,Date,Close,High,Low,Open,Volume,Ticker
0,2015-10-05,4.261959,4.346354,4.201300,4.240860,14079400,BBAS3
1,2015-10-06,4.359542,4.380641,4.174927,4.230312,15421400,BBAS3
2,2015-10-07,4.728772,4.728772,4.414927,4.414927,44423400,BBAS3
3,2015-10-08,4.660200,4.760419,4.636464,4.694485,21626400,BBAS3
4,2015-10-09,4.881738,4.881738,4.683936,4.747233,22702600,BBAS3
...,...,...,...,...,...,...,...
37000,2025-09-29,36.369999,36.970001,36.029999,36.970001,4873200,WEGE3
37001,2025-09-30,36.590000,36.790001,36.250000,36.599998,7668000,WEGE3
37002,2025-10-01,35.790001,36.360001,35.520000,36.290001,11275700,WEGE3
37003,2025-10-02,36.130001,36.349998,35.740002,35.869999,8501800,WEGE3


In [19]:
df_unico.to_csv(RAW_DATA_DIR / "tech_data.csv", index=False)